In [1]:
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt

import os
from astropy.io import fits
import astropy.units as u
import astropy
from astropy.coordinates import SkyCoord
import getpass

import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage
from lsst.daf.butler import Butler
import lsst.geom as geom

In [2]:
afwDisplay.setDefaultBackend('matplotlib')
plt.style.use('tableau-colorblind10')

In [3]:
repo = '/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo'
collections = ["LSSTCam-imSim/defaults"]
butler = Butler(repo, collections=collections)

In [4]:
raws = list(butler.registry.queryDataIds(
    ["tract", "patch", "visit", "detector"],
    instrument="LSSTCam-imSim",
    datasets="raw",
    collections=collections,
    where=f"tract=4026 AND patch=15 AND skymap='DC2'"
))

In [5]:
bands = np.unique([c['band'] for c in raws], return_counts=True)

In [6]:
bands

(array(['g', 'i', 'r', 'u', 'y', 'z'], dtype='<U1'),
 array([ 358,  964, 1007,  295,  719,  446]))

In [7]:
day_obs_402615 = np.unique([c['day_obs'] for c in raws], return_counts=True)

In [8]:
years = [int(str(x)[:4]) for x in day_obs_402615[0]]
month = [int(str(x)[4:4+2]) for x in day_obs_402615[0]]
day = [int(str(x)[6:]) for x in day_obs_402615[0]]
dates = pd.DataFrame([years, month, day,day_obs_402615[1] ]).T
dates.columns = ["year", "month", "day", "total_raws"]

In [9]:
datest_group = dates.groupby(["year", "month"]).sum()

In [10]:
for yy in np.unique(dates["year"]):
    year_20yy = dates[(dates["year"] == yy) & (dates["total_raws"] > 5) ]
    dates_20yy = pd.to_datetime(year_20yy[['year', 'month', 'day']])
    dates_20yy = np.array([str(x).replace('-', '') for x in dates_20yy.dt.date], dtype=int)
    my_days_tupleString = "("+",".join(dates_20yy.astype(str))+")"
    print(yy, my_days_tupleString)

2022 (20220103,20220108,20220111,20220117,20220219,20220730,20220806,20220811,20220813,20220818,20220827,20220829,20220831,20220909,20220912,20220914,20220916,20220917,20220928,20221001,20221004,20221006,20221008,20221009,20221013,20221014,20221015,20221016,20221017,20221024,20221026,20221103,20221114,20221122,20221128,20221201,20221202,20221215,20221218,20221221,20221223)
2023 (20230101,20230105,20230112,20230113,20230123,20230130,20230202,20230301,20230305,20230716,20230717,20230719,20230721,20230802,20230805,20230809,20230820,20230822,20230823,20230827,20230830,20230910,20230912,20230913,20230923,20230927,20230929,20231006,20231016,20231022,20231023,20231024,20231029,20231111,20231120,20231129,20231130,20231201,20231207,20231230)
2024 (20240225,20240228,20240320,20240710,20240805,20240807,20240808,20240811,20240818,20240823,20240826,20240827,20240903,20240905,20240914,20240915,20240920,20240929,20240930,20241001,20241002,20241007,20241013,20241017,20241019,20241020,20241021,20241022

In [11]:
year_2022 = dates[(dates["year"] == 2022) & (dates["total_raws"] > 5) ]

In [12]:
dates_2022 = pd.to_datetime(year_2022[['year', 'month', 'day']])

In [13]:
dates_2022 = np.array([str(x).replace('-', '') for x in dates_2022.dt.date], dtype=int)

In [14]:
my_days_tupleString = "("+",".join(dates_2022.astype(str))+")"
print(my_days_tupleString)

(20220103,20220108,20220111,20220117,20220219,20220730,20220806,20220811,20220813,20220818,20220827,20220829,20220831,20220909,20220912,20220914,20220916,20220917,20220928,20221001,20221004,20221006,20221008,20221009,20221013,20221014,20221015,20221016,20221017,20221024,20221026,20221103,20221114,20221122,20221128,20221201,20221202,20221215,20221218,20221221,20221223)


In [15]:
dates

,year,month,day,total_raws
0,2022,1,3,16
1,2022,1,8,8
2,2022,1,11,25
3,2022,1,17,29
4,2022,1,18,5
...,...,...,...,...
258,2026,12,5,1
259,2026,12,12,6
260,2026,12,16,7
261,2026,12,17,20


In [17]:
len(raws)

3789

In [20]:
day_obs_402615_all = np.array([c['day_obs'] for c in raws])
bands_all = np.array([c['band'] for c in raws])
years_all = [int(str(x)[:4]) for x in day_obs_402615_all]
month_all = [int(str(x)[4:4+2]) for x in day_obs_402615_all]
day_all = [int(str(x)[6:]) for x in day_obs_402615_all]
dates_all = pd.DataFrame([years_all, month_all, day_all, bands_all ]).T
dates_all.columns = ["year", "month", "day", "band"]
df_band = pd.get_dummies(dates_all['band'], dtype=int)
df = pd.concat([dates_all, df_band], axis=1).reindex(dates_all.index)

In [27]:
df.groupby(["year", "month"]).agg({'g':'sum', 'i':'sum', 'r':'sum', 'u':'sum', 'y':'sum', 'z':'sum'})

g   i    r   u   y   z
year month                         
2022 1       0   0    0  16  37  30
     2       0   0    0   0  13   0
     7      14   0    0   0   0   0
     8      14  22   12  24  25   0
     9       3   6   68   6  31   0
     10     29  89   73   2  33  36
     11      0  42   16   0   0  10
     12     18  16   19   0   0  15
2023 1      12  22    7  14  16   0
     2       0   0    5   0   7   0
     3       0   0    0   0   7   9
     7       6  17   37   0   4   0
     8      20  44    6   0  47  31
     9       0  19   33  32  29   0
     10      0  23   55   1  30  14
     11     39  63    0   4   4   5
     12     11  19    7   0   5   0
2024 1       0   5    0   0   1   0
     2       0   0    0   0   9   7
     3       0   0    0   0   0  14
     7       0   0    7   0   0   0
     8      17  52   58   0  18   0
     9      10   6   69  24  14  32
     10     17  65   40  40   0  12
     11      0  20    0   2  65  39
     12      0   9   28   0   0   7
2025 1       7   0    8   3   4  21
     2       7   0    0   0   0   0
     3       0   0   14   0   9   0
     7       0  19    0   0  30   0
     8       0   0   50  26   7  20
     9      12  47   23   0   0   0
     10      2  66   32   7  60  10
     11     16  20   43  17   0   5
     12     13  20   23   0   0  11
2026 1       0  39   17   0  32   0
     2      18  18   11   0  24   7
     3      13   0   15   0  31   6
     7       0   0    0   0   6   0
     8      11   0   45   0  67   0
     9       9  62    8  43  25  18
     10     15  59  144  27  19  29
     11     25  31   14   0  10  49
     12      0  44   20   7   0   9